### **SQL Interview questions & answers**

In [0]:
%sql
CREATE TABLE Orders (
  order_id INT,
  customer_id INT NOT NULL,
  order_date DATE NOT NULL,
  amount DECIMAL NOT NULL
);

In [0]:
%sql
INSERT INTO Orders VALUES
( 1, 201, '2024-01-15', 220.00),
( 2, 201, '2024-02-10', 180.00),
( 3, 201, '2024-03-12', 210.00),
( 4, 201, '2024-04-13', 200.00),
( 5, 201, '2024-05-05', 190.00),
( 6, 201, '2024-06-10', 170.00),
( 7, 201, '2024-07-14', 250.00),
( 8, 201, '2024-08-10', 260.00),
( 9, 201, '2024-09-14', 280.00),
(10, 201, '2024-10-27', 260.00),
(11, 201, '2024-11-19', 300.00),
(12, 201, '2024-12-19', 320.00),

(13, 202, '2024-03-10', 100.00),
(14, 203, '2024-03-10', 200.00),
(15, 204, '2024-03-10', 300.00),
(16, 202, '2024-07-07', 450.00),
(17, 203, '2024-07-07', 450.00),
(18, 204, '2024-07-07', 150.00),

(19, 205, '2024-01-10', 400.00),
(20, 205, '2024-02-10', 400.00),
(21, 205, '2024-03-10', 350.00),
(22, 205, '2024-04-07', 350.00),

(23, 202, '2024-01-05', 500.00),
(24, 202, '2024-04-22', 500.00),
(25, 203, '2024-02-14', 150.00),
(26, 203, '2024-04-20', 100.00),
(27, 206, '2024-02-17', 700.00),
(28, 204, '2024-02-18', 300.00),

(29, 206, '2025-07-01', 600.00),
(30, 206, '2025-08-01', 650.00),
(31, 207, '2025-09-15', 120.00);

num_affected_rows,num_inserted_rows
31,31


In [0]:
%sql
select * from Orders limit 10

order_id,customer_id,order_date,amount
1,201,2024-01-15,220
2,201,2024-02-10,180
3,201,2024-03-12,210
4,201,2024-04-13,200
5,201,2024-05-05,190
6,201,2024-06-10,170
7,201,2024-07-14,250
8,201,2024-08-10,260
9,201,2024-09-14,280
10,201,2024-10-27,260


**Customer who placed order in the last 90 days**

In [0]:
%sql
select current_date(),current_date() - interval 30 days

current_date(),current_date()-INTERVAL30DAYS
2025-11-30,2025-10-31


In [0]:
%sql
select * from Orders where order_date between current_date() and current_date() - interval 90 days

order_id,customer_id,order_date,amount


**Find customer who placed order in all months of 2024**

In [0]:
%sql
select customer_id from Orders where year(order_date)=2024 group by customer_id having count(distinct month(order_date)) =12

customer_id
201


**Find customer who placed the same order amount more than once**

In [0]:
%sql
select customer_id, amount, count(*) from Orders group by 1,2 having count(*) > 1

customer_id,amount,count(*)
201,260,2
204,300,2
205,400,2
205,350,2
202,500,2


**Get orders above the average amount for that same day**

In [0]:
%sql
select * from Orders o1 where amount > (select avg(amount) avg_amount from Orders o2 where o1.order_date=o2.order_date)

order_id,customer_id,order_date,amount
21,205,2024-03-10,350
20,205,2024-02-10,400
15,204,2024-03-10,300
17,203,2024-07-07,450
16,202,2024-07-07,450


**Find the product that always increased**

In [0]:
%sql
CREATE TABLE product_prices (
    product_id VARCHAR(10),
    price DECIMAL(10,2),
    price_date DATE
);

INSERT INTO product_prices VALUES
('P1', 100, '2024-01-01'),
('P1', 120, '2024-03-01'),
('P1', 130, '2024-05-01'),
('P2', 200, '2024-01-01'),
('P2', 210, '2024-03-01'),
('P2', 190, '2024-05-01'),
('P3', 300, '2024-01-01'),
('P3', 280, '2024-03-01'),
('P3', 260, '2024-05-01'),
('P4', 400, '2024-01-01'),
('P4', 400, '2024-03-01'),
('P4', 400, '2024-05-01');

num_affected_rows,num_inserted_rows
12,12


In [0]:
%sql
WITH cte AS (
  SELECT *, 
         LAG(price) OVER(PARTITION BY product_id ORDER BY price_date) AS prev_price 
  FROM product_prices
)
SELECT product_id, 
       price_date, 
       price, 
       CASE 
         WHEN prev_price IS NULL THEN NULL 
         WHEN price > prev_price THEN 'increased'
         WHEN price < prev_price THEN 'decreased' 
         ELSE 'no change' 
       END AS status 
FROM cte;


product_id,price_date,price,status
P1,2024-01-01,100.00,null
P1,2024-03-01,120.00,increased
P1,2024-05-01,130.00,increased
P2,2024-01-01,200.00,null
P2,2024-03-01,210.00,increased
P2,2024-05-01,190.00,decreased
P3,2024-01-01,300.00,null
P3,2024-03-01,280.00,decreased
P3,2024-05-01,260.00,decreased
P4,2024-01-01,400.00,null


In [0]:
%sql
WITH cte AS (
  SELECT *, 
         LAG(price) OVER(PARTITION BY product_id ORDER BY price_date) AS prev_price 
  FROM product_prices
), cte1 AS (
  SELECT product_id, 
         price_date, 
         price, 
         CASE 
           WHEN prev_price IS NULL THEN NULL
           WHEN price > prev_price THEN 'increased'
           WHEN price < prev_price THEN 'decreased'
           ELSE 'no change' 
         END AS status 
  FROM cte
)
SELECT product_id 
FROM cte1 
WHERE status IS NOT NULL AND status <> 'no change' GROUP BY product_id HAVING COUNT(DISTINCT status) = 1 AND MAX(status) = 'increased';


product_id
P1


### **PySpark Coding Question and Answer**

**Find missing numbers/orders in given sequence**

In [0]:
data = [(1,), (2,), (3,), (5,), (7,), (8,), (10,)]
df = spark.createDataFrame(data, ["order_id"])
display(df)

order_id
1
2
3
5
7
8
10


In [0]:
list = [(i,) for i in range(1,11)]
print(list)
df_new = spark.createDataFrame(list,['order_id'])
display(df_new)

[(1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,), (9,), (10,)]


order_id
1
2
3
4
5
6
7
8
9
10


In [0]:
df_new.subtract(df).show()

+--------+
|order_id|
+--------+
|       4|
|       6|
|       9|
+--------+



**JSON string parsing - Extract values from json**

In [0]:
data = [
    (1, '{"product":"Laptop","price":50000,"brand":"Dell"}'),
    (2, '{"product":"Phone","price":30000,"brand":"Samsung"}'),
    (3, '{"product":"Tablet","price":20000,"brand":"Apple"}')
]

df = spark.createDataFrame(data, ["id", "json_str"])
df.show(truncate=False)

+---+---------------------------------------------------+
|id |json_str                                           |
+---+---------------------------------------------------+
|1  |{"product":"Laptop","price":50000,"brand":"Dell"}  |
|2  |{"product":"Phone","price":30000,"brand":"Samsung"}|
|3  |{"product":"Tablet","price":20000,"brand":"Apple"} |
+---+---------------------------------------------------+



In [0]:
from pyspark.sql.functions import *
df.select("id",get_json_object(col("json_str"),"$.product").alias("product"),
          get_json_object(col("json_str"),"$.price").alias("price"),
          get_json_object(col("json_str"),"$.brand").alias("brand")
          ).show()

+---+-------+-----+-------+
| id|product|price|  brand|
+---+-------+-----+-------+
|  1| Laptop|50000|   Dell|
|  2|  Phone|30000|Samsung|
|  3| Tablet|20000|  Apple|
+---+-------+-----+-------+



### Python Coding Question and Answer

**Remove Duplicates**

In [0]:
l = [1, 2, 2, 3, 4, 5, 3, 4]

a = []
for i in l:
  if i not in a:
    a.append(i)
print(a)

[1, 2, 3, 4, 5]


**Sort the Array**

In [0]:
a = [1, 22, 3, 4, 55, 2]
#sorted(a)
while i < len(a)-1:
  if a[i] > a[i+1]:
    a[i], a[i+1] = a[i+1], a[i]
    i = 0
  else:
    i += 1
print(a)

[1, 2, 3, 4, 22, 55]


**Two Sum in Python**

In [0]:
l = [2, 3, 4, 7, 11, 5]
target = 7

d = {}
new = []

for i, num in enumerate(l):
  if num in d:
    new.append([d[num], i])
  else:
    d[target-num] = i
print(new)

[[1, 2], [0, 5]]


**Move Zero to End**

In [0]:
l = [1, 0, 2, 3, 4, 0]
pos = 0
for i in range(len(l)):
    if l[i]!=0:
        l[i], l[pos] = l[pos], l[i]
        pos += 1
print(l)

[1, 2, 3, 4, 0, 0]


**find second largest from the list**

In [0]:
l = [1, 0, 2, 3, 4, 0]

largest = float('-inf')
second = float('-inf')
for i in l:
    if i > largest:
        second = largest
        largest = i
    elif (largest>i) & (i>second):
        second = i
print(second)

3


**Anagram Problems**
**Input: ["eat", "tea", "tan", "ate", "nat", "bat"]
Output: [["eat", "tea", "ate"], ["tan", "nat"], ["bat"]]**

In [0]:
l = ["eat", "tea", "tan", "ate", "nat", "bat"]

from collections import defaultdict

d = defaultdict(list)

for i in l:
    d[''.join(sorted(i))].append(i)
print(d.values())

dict_values([['eat', 'tea', 'ate'], ['tan', 'nat'], ['bat']])


**Find and print the movie with the highest average rating**

In [0]:
movie_ratings = {
    "Inception": [5, 4, 5, 4, 5],
    "Interstellar": [4, 4, 5, 4, 5],
    "Tenet": [3, 3, 4, 2, 4]
}

for key,value in movie_ratings.items():
    print(key, sum(value)/len(value))

Inception 4.6
Interstellar 4.4
Tenet 3.2
